### Install Modules


Install all dependencies inside requirements.txt and other dependencies


In [ ]:
!python -m pip install -r requirements.txt

# Install the Azure Identity library
!python -m pip install azure-identity

# Install Dotenv for environment variable management
!python -m pip install python-dotenv

# Install nest_asyncio to allow nested event loops in Jupyter
!python -m pip install nest_asyncio

# Install pytz for timezone handling
!python -m pip install pytz


### Authenticate and Initialize Graph


In [ ]:
# Get the keys from .env file
import os
from dotenv import load_dotenv
load_dotenv()

# Get the client ID and secret from environment variables
tenant_id = os.getenv("AZURE_TENANT_ID")
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")
current_user_id = os.getenv("USER_ID")



calendly_client_id= os.getenv("CALENDLY_CLIENT_ID")
calendly_client_secret= os.getenv("CALENDLY_CLIENT_SECRET")
calendly_redirect_uri= os.getenv("CALENDLY_REDIRECT_URI")
calendly_webhook_secret= os.getenv("CALENDLY_WEBHOOK_SECRET")
calendly_personal_access_token= os.getenv("CALENDLY_PERSONAL_ACCESS_TOKEN")

# Display the values (redacted for security)
print(f"Client ID: {client_id[:5]}... (redacted)" if client_id else "Client ID: Not found")
print(f"Client Secret: {'*' * 8}" if client_secret else "Client Secret: Not found")
print(f"Tenant ID: {tenant_id[:5]}... (redacted)" if tenant_id else "Tenant ID: Not found")
print(f"User ID: {current_user_id[:5]}... (redacted)" if current_user_id else "User ID: Not found")

print(f"Client ID: {calendly_client_id[:5]}... (redacted)" if calendly_client_id else "Client ID: Not found")
print(f"Client Secret: {'*' * 8}" if calendly_client_secret else "Client Secret: Not found")
print(f"Redirect URI: {calendly_redirect_uri[:5]}... (redacted)" if calendly_redirect_uri else "Redirect URI: Not found")
print(f"Webhook Secret: {calendly_webhook_secret[:5]}... (redacted)" if calendly_webhook_secret else "Webhook Secret: Not found")
print(f"Personal Access Token: {calendly_personal_access_token[:5]}... (redacted)" if calendly_personal_access_token else "Personal Access Token: Not found")


### Initialize Graph Client

In [ ]:
# Configure Graph client for app-only authentication 
from azure.identity import ClientSecretCredential
from msgraph import GraphServiceClient

# Verify that environment variables are set before proceeding
if not all([client_id, client_secret, tenant_id]):
    raise ValueError("One or more required environment variables are not set. Please check your .env file.")

# Print the type of the env variables to ensure they are strings
print(f"Tenant ID: {tenant_id} (Type: {type(tenant_id)})")
print(f"Client ID: {client_id} (Type: {type(client_id)})")
print(f"Client Secret: {client_secret} (Type: {type(client_secret)})")

credential = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret
)

# Create a Graph client using the credential object
graph_client = GraphServiceClient(credential)
print("Graph client initialized successfully")

### Get Token


In [ ]:
# Function to get an access token using ClientSecretCredential
def get_token():
    try:
        # Use the default scope for Microsoft Graph
        graph_scope = "https://graph.microsoft.com/.default"
        
        # Get the token synchronously
        token = credential.get_token(graph_scope)
        return token.token
    except Exception as e:
        print(f"Error obtaining token: {e}")
        raise

### Display Token


In [ ]:
# Display the access token
def display_access_token():
    try:
        # Get the access token
        token = get_token()
        
        # Display a part of the access token for security
        if token:
            token_length = len(token)
            print(f"Access Token: {token[:25]}...{token[-5:]}")
            print(f"Token length: {token_length} characters")
            print("\nWARNING: This token contains sensitive credentials.")
            print("Do not share or commit this token to version control.")
        else:
            print("Failed to retrieve token.")
            
        return token
    except Exception as e:
        print(f"Error retrieving token: {e}")
        return None

In [ ]:
# Get and display the token
# token = display_access_token()

In [ ]:
# Display the raw token (for debugging purposes)
def display_raw_token():
    try:
        token = get_token()
        print(token)
        print(f"\nRaw token displayed. Length: {len(token)} characters")
        return token
    except Exception as e:
        print(f"Error displaying raw token: {e}")
        return None

In [ ]:
# Uncomment the line below to see the full raw token (use with caution)
# raw_token = display_raw_token()

### List Users


Get users v1

In [ ]:
import asyncio

async def get_users_async():
    try:
        # Get the list of users with the async approach
        users_response = await graph_client.users.get()
        
        # Print the user details
        print(f"Retrieved {len(users_response.value)} users")
        for user in users_response.value:
            # Use proper property access for user objects
            display_name = user.display_name if hasattr(user, 'display_name') else 'No display name'
            email = user.mail if hasattr(user, 'mail') and user.mail else user.user_principal_name
            print(f"User ID: {user.id}, Name: {display_name}, Email: {email}")
        
        return users_response.value
    except Exception as e:
        print(f"Error retrieving users: {e}")
        import traceback
        traceback.print_exc()
        return None

Get users v2

In [ ]:
from msgraph.generated.users.users_request_builder import UsersRequestBuilder

async def get_users_async_v2():
    try:
        # Set the top limit for the number of users to retrieve
        topLimit = 50

        # Build query parameters for Users Request Builder
        query_params = UsersRequestBuilder.UsersRequestBuilderGetQueryParameters(
            select = ['displayName', 'id', 'mail'],
            top = topLimit,
            orderby = ['displayName desc']
        )

        # create a request_config object
        request_config = UsersRequestBuilder.UsersRequestBuilderGetRequestConfiguration(
            query_parameters=query_params
        )

        # Get the list of users with the async approach
        users_response = await graph_client.users.get(request_configuration=request_config)
        
        # Print the user details (limited to 10 for display purposes)
        count = 0
        for user in users_response.value:
            print(f"User ID: {user.id}, Display Name: {user.display_name}, Email: {user.mail or user.user_principal_name}")
            count += 1
            if count >= topLimit:
                break
        
        return users_response.value
    except Exception as e:
        print(f"Error retrieving users: {e}")
        return None

In [ ]:
# Import required libraries
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow running asyncio in Jupyter notebook
nest_asyncio.apply()

# Display Users using async execution
try:
    # Use asyncio directly now that we've patched it with nest_asyncio
    loop = asyncio.get_event_loop()

    users = loop.run_until_complete(get_users_async())
    # users = loop.run_until_complete(get_users_async_v2())

    print(f"\nTotal users retrieved: {len(users) if users else 0}")
except Exception as e:
    print(f"Error executing async function: {e}")
    
    import traceback
    traceback.print_exc()

Tabulate Data for msgraph calendar api


In [ ]:
from tabulate import tabulate

def convert_to_pst(date_time_str):
    utc_timezone = pytz.timezone("UTC")
    dt = datetime.datetime.fromisoformat(date_time_str)
    dt_utc = utc_timezone.localize(dt)
    pst_timezone = pytz.timezone('America/Los_Angeles')
    dt_pst = dt_utc.astimezone(pst_timezone)
    return dt_pst

def tabulate_calendar_events(events):
    table_data = []
    for i, event in enumerate(events, 1):
        subject = event.subject or "(No subject)"
        organizer = event.organizer.email_address.name if event.organizer and hasattr(event.organizer, 'email_address') else "Unknown"

        start_time_dt_pst = convert_to_pst(event.start.date_time)
        start_time = start_time_dt_pst.strftime("%Y-%m-%d %H:%M:%S %Z")

        end_time_dt_pst = convert_to_pst(event.end.date_time)
        end_time = end_time_dt_pst.strftime("%Y-%m-%d %H:%M:%S %Z")

        location = event.location.display_name if event.location else "No location"
        is_all_day = "Yes" if event.is_all_day else "No"
        importance = event.importance.capitalize() if event.importance and event.importance != 'normal' else "Normal"

        attendees = ", ".join(
            f"{a.email_address.name} ({a.email_address.address})"
            for a in event.attendees
        ) if hasattr(event, 'attendees') and event.attendees else "None"

        table_data.append([
            i, subject, organizer, start_time, end_time,
            location, is_all_day, importance, attendees
        ])

    headers = ["#", "Subject", "Organizer", "Start Time", "End Time", "Location", "All Day", "Importance", "Attendees"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))


### Get Calendar View

Retrieve calendar events within a specific time range


In [ ]:
import datetime
import pytz
from msgraph.generated.users.item.calendar.calendar_view.calendar_view_request_builder import CalendarViewRequestBuilder

async def get_calendar_view(user_id=None):
    try:
        # Create datetime objects for the date range
        now = datetime.datetime.now(pytz.timezone("America/Los_Angeles"))
        start_date = now
        end_date = pytz.timezone("America/Los_Angeles").localize(datetime.datetime(2026, 12, 20, 23, 59, 59)) #(year, month, day, hour, minute, second)
        
        # Create query parameters for the calendar view request
        query_parameters = CalendarViewRequestBuilder.CalendarViewRequestBuilderGetQueryParameters(
            start_date_time=start_date,
            end_date_time=end_date,
            select=['subject', 'organizer', 'start', 'end', 'isAllDay', 'location', 'bodyPreview', 'importance', 'attendees'],
            orderby=['start/dateTime asc']
        )

        # Check if we're getting calendar for a specific user or the current user
        print(f"Fetching calendar events for user {user_id} from {start_date} to {end_date}")
        
        # For specific user
        # Step 1: Create request parameters
        request_config = CalendarViewRequestBuilder.CalendarViewRequestBuilderGetRequestConfiguration(
            query_parameters=query_parameters
        )

        # Step 3: Make the call
        calendar_view = await graph_client.users.by_user_id(user_id).calendar.calendar_view.get(request_configuration=request_config)
                
        # Process and display the events
        if hasattr(calendar_view, 'value') and calendar_view.value:
            print(f"Retrieved {len(calendar_view.value)} calendar events.")
            tabulate_calendar_events(calendar_view.value)
        else:
            print("No events found in the specified time range.")
        return calendar_view.value if hasattr(calendar_view, 'value') else []
    except Exception as e:
        print(f"Error retrieving calendar view: {e}")
        import traceback
        traceback.print_exc()
        return None

Fetch all calendar events within date range

In [ ]:
from kiota_abstractions.request_information import RequestInformation
from kiota_abstractions.method import Method
from msgraph.generated.models.event_collection_response import EventCollectionResponse
from msgraph.generated.users.item.calendar.calendar_view.calendar_view_request_builder import CalendarViewRequestBuilder

async def fetch_all_calendar_events(start_date, end_date, select_fields=None, orderby=None):
    all_events = []
    query_parameters = CalendarViewRequestBuilder.CalendarViewRequestBuilderGetQueryParameters(
        start_date_time=start_date,
        end_date_time=end_date,
        select=select_fields,
        orderby=orderby
    )

    request_config = CalendarViewRequestBuilder.CalendarViewRequestBuilderGetRequestConfiguration(
        query_parameters=query_parameters
    )

    request_builder = graph_client.users.by_user_id(current_user_id).calendar.calendar_view
    response = await request_builder.get(request_configuration=request_config)
    events = getattr(response, 'value', [])
    all_events.extend(events)

    # Pagination
    next_link = getattr(response, 'odata_next_link', None) or getattr(response, '@odata.nextLink', None)
    request_adapter = graph_client.request_adapter  # Use the public adapter property

    while next_link:
        from msgraph.generated.models.event_collection_response import EventCollectionResponse
        request_info = RequestInformation()
        request_info.url = next_link
        request_info.http_method = Method.GET
        request_info.headers.add("Accept", "application/json")

        next_response = await request_adapter.send_async(
            request_info,
            EventCollectionResponse,
            {}
        )
        events = getattr(next_response, 'value', [])
        all_events.extend(events)
        next_link = getattr(next_response, 'odata_next_link', None) or getattr(next_response, '@odata.nextLink', None)

    return all_events


Display Calendar Events

In [ ]:
# Import required libraries
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow running asyncio in Jupyter notebook
nest_asyncio.apply()

try:
    # Use asyncio directly now that we've patched it with nest_asyncio
    loop = asyncio.get_event_loop()

    # Using the nest_asyncio approach we established earlier
    user_calendar_events = loop.run_until_complete(get_calendar_view())
    
    # Summary of results
    # if user_calendar_events:
    #     print(f"\nSummary: Retrieved {len(user_calendar_events)} calendar events for user {current_user_id}.")
    # else:
    #     print(f"No calendar events were retrieved for user {current_user_id} or an error occurred.")
except Exception as e:
    print(f"Error executing calendar view function for specific user: {e}")
    import traceback
    traceback.print_exc()

Display Calendar All Calendar Events in Date Range

In [ ]:
import datetime
import pytz

# Set your timezone and date range
tz = pytz.timezone("America/Los_Angeles")
now = datetime.datetime.now(tz)
start_date = now
end_date = tz.localize(datetime.datetime(2025, 6, 5, 23, 59, 59)) #(year, month, day, hour, minute, second)

select_fields = ['subject', 'organizer', 'start', 'end', 'isAllDay', 'location', 'bodyPreview', 'importance', 'attendees']
orderby = ['start/dateTime asc']

try:
    loop = asyncio.get_event_loop()
    all_calendar_events = loop.run_until_complete(
        fetch_all_calendar_events(
            start_date,
            end_date,
            select_fields,
            orderby
        )
    )
    if all_calendar_events:
        print(f"Retrieved {len(all_calendar_events)} calendar events.")
        tabulate_calendar_events(all_calendar_events)
except Exception as e:
    print(f"Error executing calendar view function for specific user: {e}")
    import traceback
    traceback.print_exc()

Filter fetched Calendar Events

In [ ]:
def filter_and_display_events(events, limit=None, organizer=None, subject=None):
    """
    Filter and display events based on optional criteria.
    - events: list of fetched event objects
    - limit: int, number of events to display (e.g., 5 for first 5)
    - organizer: str, display only events where organizer matches (case-insensitive)
    - subject: str, display only events where subject contains this string (case-insensitive)
    """
    filtered = events

    # Filter by organizer name if provided
    if organizer:
        filtered = [
            e for e in filtered
            if hasattr(e, "organizer")
            and hasattr(e.organizer, "email_address")
            and organizer.lower() in (getattr(e.organizer.email_address, "name", "") or "").lower()
        ]

    # Filter by subject if provided
    if subject:
        filtered = [
            e for e in filtered
            if hasattr(e, "subject") and subject.lower() in (e.subject or "").lower()
        ]

    # Apply limit if provided
    if limit is not None:
        filtered = filtered[:limit]

    # Display the filtered events
    tabulate_calendar_events(filtered)
    print(f"\nDisplayed {len(filtered)} event(s) out of {len(events)} fetched.")

# Example usage:
# Show first 5 events
filter_and_display_events(all_calendar_events, limit=5) # limit is optional

# Show events organized by "GoHealth UC"
filter_and_display_events(all_calendar_events, organizer="GoHealth UC") # organizer name is case-insensitive

# Show events with subject containing "meeting"
filter_and_display_events(all_calendar_events, subject="meeting") # subject is case-insensitive

Clean Create Event function response

In [ ]:
import re
from bs4 import BeautifulSoup

def clean_body_response(html_content: str) -> dict:
    """
    Extracts important parts from the HTML email body content.
    Returns a dictionary with the main description, meeting ID, and passcode.
    """
    soup = BeautifulSoup(html_content, "html.parser")
    text = soup.get_text(separator="\n")

    # Normalize excessive newlines and trim
    text = re.sub(r'\n\s*\n+', '\n', text).strip()

    details = {
        "description": "",
        "meeting_id": "",
        "passcode": ""
    }

    # Extract description before the Microsoft Teams section
    split_text = text.split("Microsoft Teams")
    details["description"] = split_text[0].strip() if split_text else ""

    # Extract meeting ID
    match = re.search(r"Meeting ID:\s*(\d[\d\s]*)", text)
    if match:
        details["meeting_id"] = match.group(1).strip()

    # Extract passcode
    match = re.search(r"Passcode:\s*(\S+)", text)
    if match:
        details["passcode"] = match.group(1).strip()

    return details


Validate Create calendar inputs

In [ ]:
from datetime import datetime
from msgraph.generated.models.recurrence_pattern_type import RecurrencePatternType

def validate_event_input(
    subject: str,
    start_datetime: datetime,
    end_datetime: datetime,
    attendees: list = None,
    recurrence: dict = None
):
    if not subject or not isinstance(subject, str):
        raise ValueError("Subject must be a non-empty string.")

    if not isinstance(start_datetime, datetime) or not isinstance(end_datetime, datetime):
        raise ValueError("Start and end datetime must be datetime objects.")

    if start_datetime >= end_datetime:
        raise ValueError("Start datetime must be before end datetime.")

    if attendees:
        for email in attendees:
            if not isinstance(email, str) or '@' not in email:
                raise ValueError(f"Invalid email format: {email}")

    if recurrence:
        valid_types = {e.value for e in RecurrencePatternType}
        recurrence_type = recurrence.get('type', 'daily')
        if recurrence_type not in valid_types:
            raise ValueError(f"Invalid recurrence type: {recurrence_type}. Must be one of {valid_types}")

        range_type = recurrence.get('range_type', 'noEnd')
        if range_type == 'endDate' and 'end_date' not in recurrence:
            raise ValueError("Missing 'end_date' for recurrence with range_type 'endDate'.")
        if range_type == 'numbered' and 'occurrences' not in recurrence:
            raise ValueError("Missing 'occurrences' for recurrence with range_type 'numbered'.")


Create Event Function

In [ ]:
from datetime import datetime
import pytz
from msgraph.generated.models.event import Event
from msgraph.generated.models.item_body import ItemBody
from msgraph.generated.models.body_type import BodyType
from msgraph.generated.models.date_time_time_zone import DateTimeTimeZone
from msgraph.generated.models.attendee import Attendee
from msgraph.generated.models.email_address import EmailAddress
from msgraph.generated.models.location import Location
from msgraph.generated.models.patterned_recurrence import PatternedRecurrence
from msgraph.generated.models.recurrence_pattern import RecurrencePattern
from msgraph.generated.models.recurrence_range import RecurrenceRange
from msgraph.generated.models.recurrence_pattern_type import RecurrencePatternType
from msgraph.generated.models.importance import Importance
from msgraph.generated.models.response_status import ResponseStatus

async def create_calendar_event(
    user_id: str,
    subject: str,
    content: str,
    start_datetime: datetime,
    end_datetime: datetime,
    timezone: str = 'Asia/Manila',
    is_online_meeting: bool = False,
    attendees: list = None,
    location: str = None,
    is_all_day: bool = False,
    importance: Importance = Importance.Normal,
    recurrence: dict = None
):
    """
    Create a new calendar event for the specified user (application-level).
    """
    try:
        if not start_datetime.tzinfo:
            start_datetime = pytz.timezone(timezone).localize(start_datetime)
        if not end_datetime.tzinfo:
            end_datetime = pytz.timezone(timezone).localize(end_datetime)

        event = Event()
        event.subject = subject

        event.body = ItemBody()
        event.body.content_type = BodyType.Html
        event.body.content = content

        event.start = DateTimeTimeZone()
        event.start.date_time = start_datetime.isoformat()
        event.start.time_zone = "Pacific Standard Time"

        event.end = DateTimeTimeZone()
        event.end.date_time = end_datetime.isoformat()
        event.end.time_zone = "Pacific Standard Time"

        event.is_all_day = is_all_day
        event.importance = importance

        if location:
            event.location = Location()
            event.location.display_name = location

        if attendees:
            event.attendees = []
            for email in attendees:
                attendee = Attendee()
                attendee.email_address = EmailAddress(address=email)
                attendee.status = ResponseStatus(response="none")
                event.attendees.append(attendee)

        event.is_online_meeting = is_online_meeting
        if is_online_meeting:
            event.online_meeting_provider = "teamsForBusiness"

        if recurrence:
            pattern = RecurrencePattern()
            pattern.type_ = RecurrencePatternType(recurrence.get('type', 'daily'))
            pattern.interval = recurrence.get('interval', 1)
            if pattern.type_ == RecurrencePatternType.Weekly:
                pattern.days_of_week = recurrence.get('days_of_week', [])

            range_ = RecurrenceRange()
            range_.type_ = recurrence.get('range_type', 'noEnd')
            range_.start_date = start_datetime.date()

            if range_.type_ == 'endDate':
                range_.end_date = recurrence.get('end_date')
            elif range_.type_ == 'numbered':
                range_.number_of_occurrences = recurrence.get('occurrences', 5)

            event.recurrence = PatternedRecurrence(pattern=pattern, range=range_)

        created_event = await graph_client.users.by_user_id(user_id).events.post(event)
        print(f"Event created successfully: {created_event.id}")
        print(f"Web link: {created_event.web_link}")
        print(f"Subject: {created_event.subject}")
        event_summary = clean_body_response(created_event.body.content if created_event.body else "") 
        print(f"Description: {event_summary['description']}")
        print(f"Meeting ID: {event_summary['meeting_id']}")
        print(f"Passcode: {event_summary['passcode']}")
        print(f"Start: {created_event.start.date_time} ({created_event.start.time_zone})")
        print(f"End: {created_event.end.date_time} ({created_event.end.time_zone})")
        print(f"Location: {created_event.location.display_name if created_event.location else 'None'}")
        print(f"All Day: {'Yes' if created_event.is_all_day else 'No'}")
        print(f"Importance: {created_event.importance}")
        print("Attendees:")

        if created_event.attendees:
            for attendee in created_event.attendees:
                name = attendee.email_address.name if attendee.email_address.name else attendee.email_address.address
                email = attendee.email_address.address
                print(f"  - {name} ({email})")
        else:
            print("  None")

        if created_event.recurrence:
            print("Recurrence:")
            print(f"  Pattern Type: {created_event.recurrence.pattern.type}")
            print(f"  Interval: {created_event.recurrence.pattern.interval}")
            if created_event.recurrence.pattern.days_of_week:
                print(f"  Days of Week: {', '.join(created_event.recurrence.pattern.days_of_week)}")
            print(f"  Range Type: {created_event.recurrence.range.type}")
            print(f"  Start Date: {created_event.recurrence.range.start_date}")
            if created_event.recurrence.range.end_date:
                print(f"  End Date: {created_event.recurrence.range.end_date}")
            if created_event.recurrence.range.number_of_occurrences:
                print(f"  Occurrences: {created_event.recurrence.range.number_of_occurrences}")

        return created_event

    except Exception as e:
        print(f"Error creating event: {e}")
        import traceback
        traceback.print_exc()
        return None


Create the actual event

In [ ]:
import asyncio
import nest_asyncio
from datetime import datetime, timedelta

# Apply nest_asyncio to enable asyncio in Jupyter
nest_asyncio.apply()

# Create a test event
tz = pytz.timezone("Asia/Manila")
now = datetime.now(tz)
start_time = tz.localize(datetime(now.year, now.month, now.day, 16, 0)) #4pm
end_time = start_time + timedelta(hours=1)

#This is a sample payload for creating 1 instance event
event_data = {
    "subject": "Test create event function june 2",
    "content": "<p>This is a test meeting for creating event</p>",
    "start_datetime": start_time,
    "end_datetime": end_time,
    "is_online_meeting": True,
    "attendees": [""],
    "location": "Jairosoft",
    "importance": "Normal",
    "recurrence": None,
    "user_id": current_user_id  # replace with actual user ID
}

event_data_recurring ={
    "subject": "Daily Standup Meeting",
    "content": "<p>Daily team standup</p>",
    "start_datetime": start_time,
    "end_datetime": end_time,
    "is_online_meeting": True,
    "attendees": [""],
    "location": "Conference Room",
    "importance": "Normal",
    "recurrence": {
        "type": "daily",
        "interval": 1,
        "range_type": "endDate",
        "end_date": datetime(2025, 6, 4).date()
    },
    "user_id": current_user_id  # Replace with actual user ID
}

validate_event_input(
    subject=event_data["subject"],
    start_datetime=event_data["start_datetime"],
    end_datetime=event_data["end_datetime"],
    attendees=event_data.get("attendees"),
    recurrence=event_data.get("recurrence")
)

#event = await create_calendar_event(**event_data) #Uncomment this line to create a 1 instance event

event = await create_calendar_event(**event_data_recurring) #Uncomment this line to create a recurring event

Get Calendly Events


In [ ]:
from tabulate import tabulate
import requests

headers = {
    "Authorization": f"Bearer {calendly_personal_access_token}",
    "Content-Type": "application/json"
}

me = requests.get("https://api.calendly.com/users/me", headers=headers).json()
user_uri = me["resource"]["uri"]

params = {
    "user": user_uri,
    "status": "active"
}

response = requests.get("https://api.calendly.com/scheduled_events", headers=headers, params=params)
events = response.json()["collection"]

if not events:
    print("No active scheduled events found.")
else:
    table_data = []
    for i, event in enumerate(events, 1):
        event_name = event["name"]
        start_time = event["start_time"]
        end_time = event["end_time"]
        event_status = event["status"]
        event_uri = event["uri"]
        event_type_uri = event["event_type"]

        event_uuid = event_uri.split("/")[-1]
        invitees_url = f"https://api.calendly.com/scheduled_events/{event_uuid}/invitees"
        invitees_res = requests.get(invitees_url, headers=headers)
        invitees = invitees_res.json().get("collection", [])

        emails = ", ".join(
            invitee.get("email") for invitee in invitees if invitee.get("email")
        ) or "None"

        table_data.append([
            i, event_name, start_time, end_time, event_status,
            event_uri, event_type_uri, emails
        ])

    headers = [
        "#", "Event", "Start Time", "End Time", "Status",
        "URI", "Event Type", "Attendees"
    ]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))


Create Contact

In [ ]:
import requests

access_token = get_token()

url = f"https://graph.microsoft.com/v1.0/users/{current_user_id}/contacts"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

data = {
    "givenName": "Jane",
    "surname": "Doe",
    "emailAddresses": [
        {
            "address": "jane.doe@example.com",
            "name": "Jane Doe"
        }
    ],
    "businessPhones": ["+1 555-123-4567"],
    "companyName": "Contoso Ltd.",
    "jobTitle": "Senior Software Engineer"
}

response = requests.post(url, headers=headers, json=data)

print(response.status_code, response.json())


Update User contact:

In [ ]:
import requests

access_token = get_token()
contact_id = "AAMkAGQ2NjY4NmVlLTdmMzMtNGZlOS1hNDQ3LTEzY2I0YjQxZTA1ZgBGAAAAAAC35WQYyx5JRY-1fEsj-DbmBwCGhOEhmGcuSLv93UHjDGgGAAAAAAEOAACGhOEhmGcuSLv93UHjDGgGAADlcFBwAAA="

url = f"https://graph.microsoft.com/v1.0/users/{current_user_id}/contacts/{contact_id}"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

data = {
    "jobTitle": "Principal Engineer",
    "companyName": "New Contoso Inc.",
    "businessPhones": ["+1 555-999-9999"]
}

response = requests.patch(url, headers=headers, json=data)

print(response.status_code, response.text)


Read Contacts

In [ ]:
import requests
from tabulate import tabulate

def get_user_contacts(access_token, current_user_id):
    url = f"https://graph.microsoft.com/v1.0/users/{current_user_id}/contacts"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json().get("value", [])

def tabulate_user_contacts(contacts):
    table_data = []

    for i, contact in enumerate(contacts, 1):
        # contact_id = contact.get("id", "N/A")   #uncomment to display ID
        name = contact.get("displayName", "(No name)")
        email = ", ".join(e["address"] for e in contact.get("emailAddresses", [])) or "None"
        phone = ", ".join(contact.get("businessPhones", [])) or "None"
        company = contact.get("companyName") or "None"
        job_title = contact.get("jobTitle") or "None"

        #table_data.append([i, contact_id, name, email, phone, company, job_title]) #uncomment to display ID
    
        table_data.append([i, name, email, phone, company, job_title])

    #headers = ["#", "Contact ID", "Name", "Email", "Phone", "Company", "Job Title"] #uncomment to display ID
    headers = ["#", "Name", "Email", "Phone", "Company", "Job Title"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

access_token = get_token()
contacts = get_user_contacts(access_token, current_user_id)
tabulate_user_contacts(contacts)


Fetch Tasks 

In [1]:
from tabulate import tabulate
import requests

# 1. Get access token
def get_access_token():
    url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    data = {
        "client_id": client_id,
        "scope": "https://graph.microsoft.com/.default",
        "client_secret": client_secret,
        "grant_type": "client_credentials"
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    return response.json()["access_token"]

def get_task_lists(access_token):
    url = f"https://graph.microsoft.com/v1.0/users/{current_user_id}/todo/lists"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json().get("value", [])

def get_tasks_from_list(access_token, list_id):
    url = f"https://graph.microsoft.com/v1.0/users/{current_user_id}/todo/lists/{list_id}/tasks"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json().get("value", [])

# Run
token = get_access_token()
lists = get_task_lists(token)

rows = []
for l in lists:
    tasks = get_tasks_from_list(token, l['id'])
    for t in tasks:
        rows.append({
            "List": l["displayName"],
            "Title": t.get("title"),
            "Status": t.get("status")
        })

print(f"Tasks for user: {current_user_id}\n")
print(tabulate(rows, headers="keys", tablefmt="grid"))


NameError: name 'tenant_id' is not defined